In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Valor Cimento Portland,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Consumo de Cimento (t)
0,2003,226.842795,8.256606,0.797768,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,3.814993,1.320065e+08,6.638363e+06,19.725168,1.221860e+08,1687.172000
1,2004,221.696363,8.186110,0.798794,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,3.888741,1.340197e+08,6.682964e+06,19.810475,1.229692e+08,1740.105000
2,2005,223.704012,8.161534,0.799746,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,3.845774,1.358571e+08,6.722583e+06,19.879681,1.235946e+08,1824.650000
3,2006,265.456919,8.074449,0.800627,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,3.833852,1.375869e+08,6.771741e+06,19.950366,1.241926e+08,1870.478000
4,2007,322.873579,8.049751,0.801440,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,3.891640,1.391380e+08,6.821246e+06,20.007428,1.246727e+08,2081.839000
5,2008,354.241699,8.020915,0.802185,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,3.909333,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,2653.587000
6,2009,431.347926,8.049936,0.802866,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,3.988731,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,2669.400000
7,2010,792.441715,8.013741,0.803485,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,4.193595,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,2852.759000
8,2011,1268.083588,8.000656,0.804044,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,4.383085,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,3087.810000
9,2012,1549.260439,4.760238,0.804247,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,4.639898,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,3268.310000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Valor Cimento Portland,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,226.842795,8.256606,0.797768,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,3.814993,1.320065e+08,6.638363e+06,19.725168,1.221860e+08
1,221.696363,8.186110,0.798794,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,3.888741,1.340197e+08,6.682964e+06,19.810475,1.229692e+08
2,223.704012,8.161534,0.799746,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,3.845774,1.358571e+08,6.722583e+06,19.879681,1.235946e+08
3,265.456919,8.074449,0.800627,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,3.833852,1.375869e+08,6.771741e+06,19.950366,1.241926e+08
4,322.873579,8.049751,0.801440,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,3.891640,1.391380e+08,6.821246e+06,20.007428,1.246727e+08
5,354.241699,8.020915,0.802185,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,3.909333,1.404443e+08,6.811481e+06,20.042997,1.249726e+08
6,431.347926,8.049936,0.802866,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,3.988731,1.414019e+08,6.842789e+06,20.044049,1.250753e+08
7,792.441715,8.013741,0.803485,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,4.193595,1.450816e+08,6.897548e+06,20.092849,1.259937e+08
8,1268.083588,8.000656,0.804044,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,4.383085,1.483956e+08,6.888268e+06,20.118967,1.266496e+08
9,1549.260439,4.760238,0.804247,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,4.639898,1.513674e+08,6.873501e+06,20.117079,1.271489e+08


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1740.105000
1     1824.650000
2     1870.478000
3     2081.839000
4     2653.587000
5     2669.400000
6     2852.759000
7     3087.810000
8     3268.310000
9     3466.227000
10    3246.749286
11    3017.084229
12    2776.217486
13    2891.489000
14    2976.485000
15    3258.027000
16    3537.032000
17    4089.333000
18    4255.808000
19            NaN
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Valor Cimento Portland,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-1.244844,1.052493,-1.994285,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-0.991885,-1.620438,-2.186864,-2.677515,-2.020278
1,-1.253591,1.000393,-1.629028,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-0.921590,-1.410301,-1.538165,-1.877756,-1.604579
2,-1.250179,0.982230,-1.289984,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-0.962546,-1.218502,-0.961918,-1.228940,-1.272609
3,-1.179217,0.917871,-0.976329,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.973910,-1.037936,-0.246922,-0.566262,-0.955143
4,-1.081632,0.899618,-0.687238,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,-0.918827,-0.876030,0.473105,-0.031300,-0.700315
5,-1.028320,0.878307,-0.421888,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,-0.901962,-0.739679,0.331079,0.302166,-0.541142
6,-0.897272,0.899755,-0.179454,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,-0.826282,-0.639717,0.786439,0.312029,-0.486628
7,-0.283564,0.873005,0.040888,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,-0.631009,-0.255618,1.582900,0.769536,0.000860
8,0.524827,0.863335,0.239962,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,-0.450391,0.090305,1.447916,1.014396,0.349023
9,1.002709,-1.531473,0.312322,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,-0.205601,0.400513,1.233137,0.996692,0.614069


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     1740.105000
1     1824.650000
2     1870.478000
3     2081.839000
4     2653.587000
5     2669.400000
6     2852.759000
7     3087.810000
8     3268.310000
9     3466.227000
10    3246.749286
11    3017.084229
12    2776.217486
13    2891.489000
14    2976.485000
15    3258.027000
16    3537.032000
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Valor Cimento Portland,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
17,1.271362,-1.092682,-0.11741,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.917628,0.886688,-0.271847,-0.02587,0.728476


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    4089.333
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [13]:
train_input.iloc[-(len(train_input)//5):]

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Valor Cimento Portland,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
14,0.805203,-0.965102,1.300358,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,1.403991,1.217713,-0.707742,0.344885,1.039726
15,0.856363,-1.045743,0.977481,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,1.499810,1.214451,-0.532297,0.389403,1.041875
16,1.076131,-1.100971,0.560529,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,1.710664,1.131669,-0.416908,0.288784,0.974515


In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [17]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[1861477809, 1694249642, 2686832177, 1660227673, 2035459143, 657701897, 2209231, 82789410, 34030966, 1708758530, 2564910860, 3378720697, 2011867004, 3499464136, 3160238356, 250232831, 3809987463, 2371264824, 2928487607, 1588743446, 1264523767, 29342970, 1094217975, 3561593117, 1806037824, 560816226, 2640532531, 2463407718, 2605504741, 3485953676, 1955147448, 3182227189, 3861720404, 2783361713, 3316761514, 3647765541, 3691036079, 3799950766, 2534321743, 1642456668, 3841610445, 3369096463, 1066056663, 208580007, 2552960181, 810708833, 1493363177, 3316679925, 2907692913, 1804109724, 984112812, 435307829, 2538245366, 1806828957, 2374653734, 2932913612, 3021048223, 1893053392, 2183968325, 1485365009, 1852331985, 681895436, 3812171220, 3923993501, 1093989530, 1660150195, 3813709273, 2190202698, 3170896031, 1622591887, 2471297044, 217700410, 1840960962, 3738598673, 2307618235, 4112741036, 2217375533, 1567047524, 3104025497, 462049042, 1085824035, 368096669, 4137566588, 35113741, 1408274309, 2

val_loss: 54470.42578125


Step: 89 ___________________________________________
val_loss: 52792.484375


Step: 90 ___________________________________________
val_loss: 53523.96484375


Step: 91 ___________________________________________
val_loss: 52369.78125


Step: 92 ___________________________________________
val_loss: 70128.4296875


Step: 93 ___________________________________________
val_loss: 52512.09765625


Step: 94 ___________________________________________
val_loss: 51070.703125


Step: 95 ___________________________________________
val_loss: 43369.69140625


Step: 96 ___________________________________________
val_loss: 41666.15234375


Step: 97 ___________________________________________
val_loss: 159164.390625


Step: 98 ___________________________________________
val_loss: 52369.84765625


Step: 99 ___________________________________________
val_loss: 52520.57421875


final_seed: 1852331985


In [18]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 453ms/step - loss: 8024330.5000 - val_loss: 10238277.0000
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 7672483.5000 - val_loss: 1333121.0000
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 10349120.0000 - val_loss: 19213270.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 8277409.0000 - val_loss: 7005566.0000
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 7140798.5000 - val_loss: 8302844.0000
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 5555214.5000 - val_loss: 8880793.0000
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 5173726.0000 - val_loss: 8798073.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 4775048.5000 - val_loss: 8524140.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 5209792.0000 - val_loss: 8

1/1 [==============================] - 0s 13ms/step - loss: 1528048.6250 - val_loss: 1821354.6250
Epoch 74/10000
1/1 [==============================] - 0s 13ms/step - loss: 1207357.1250 - val_loss: 1580578.6250
Epoch 75/10000
1/1 [==============================] - 0s 13ms/step - loss: 1049536.5000 - val_loss: 1345761.6250
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430060.1250 - val_loss: 1304585.3750
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 1374811.0000 - val_loss: 1276550.5000
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 1404429.3750 - val_loss: 2022251.6250
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 1212191.5000 - val_loss: 1667565.0000
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 1262140.3750 - val_loss: 1470672.8750
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 1344184.1250 - val_loss: 1730696.500

Epoch 146/10000
1/1 [==============================] - 0s 13ms/step - loss: 543500.3750 - val_loss: 787345.4375
Epoch 147/10000
1/1 [==============================] - 0s 13ms/step - loss: 599616.6250 - val_loss: 990184.1875
Epoch 148/10000
1/1 [==============================] - 0s 13ms/step - loss: 531172.7500 - val_loss: 860706.8125
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 562201.5000 - val_loss: 848206.9375
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 493807.5938 - val_loss: 887105.8125
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 561781.8125 - val_loss: 845290.0000
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 524553.1250 - val_loss: 891787.5625
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 536113.5000 - val_loss: 818519.4375
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 456045.1562 - val_loss: 7773

1/1 [==============================] - 0s 14ms/step - loss: 223930.8594 - val_loss: 345511.3125
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 252311.2656 - val_loss: 342071.8438
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 202445.2031 - val_loss: 335408.7500
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 209054.3125 - val_loss: 299092.9062
Epoch 223/10000
1/1 [==============================] - 0s 13ms/step - loss: 212909.0000 - val_loss: 314749.7812
Epoch 224/10000
1/1 [==============================] - 0s 13ms/step - loss: 194678.0469 - val_loss: 313066.9688
Epoch 225/10000
1/1 [==============================] - 0s 13ms/step - loss: 203950.5938 - val_loss: 275274.4688
Epoch 226/10000
1/1 [==============================] - 0s 14ms/step - loss: 199298.3750 - val_loss: 304116.2812
Epoch 227/10000
1/1 [==============================] - 0s 14ms/step - loss: 170432.0312 - val_loss: 266575.3125
Epoch 22

1/1 [==============================] - 0s 13ms/step - loss: 101746.7969 - val_loss: 193835.2344
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 150377.6875 - val_loss: 217296.3281
Epoch 294/10000
1/1 [==============================] - 0s 13ms/step - loss: 75772.6641 - val_loss: 223141.9375
Epoch 295/10000
1/1 [==============================] - 0s 13ms/step - loss: 188704.9375 - val_loss: 229195.3750
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 69533.5078 - val_loss: 227083.6719
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 110409.8359 - val_loss: 243754.4219
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 113559.4531 - val_loss: 198732.3281
Epoch 299/10000
1/1 [==============================] - 0s 13ms/step - loss: 101910.8203 - val_loss: 175575.9375
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 98975.7734 - val_loss: 165059.9062
Epoch 301/1

...

Epoch 1105/10000
1/1 [==============================] - 0s 14ms/step - loss: 32393.5840 - val_loss: 19174.6055
Epoch 1106/10000
1/1 [==============================] - 0s 14ms/step - loss: 31438.6504 - val_loss: 18788.2324
Epoch 1107/10000
1/1 [==============================] - 0s 13ms/step - loss: 30738.8867 - val_loss: 21771.5879
Epoch 1108/10000
1/1 [==============================] - 0s 13ms/step - loss: 32899.6094 - val_loss: 20094.0723
Epoch 1109/10000
1/1 [==============================] - 0s 13ms/step - loss: 38315.2578 - val_loss: 23234.7129
Epoch 1110/10000
1/1 [==============================] - 0s 13ms/step - loss: 41308.7188 - val_loss: 21901.8828
Epoch 1111/10000
1/1 [==============================] - 0s 14ms/step - loss: 32670.8496 - val_loss: 20460.7910
Epoch 1112/10000
1/1 [==============================] - 0s 14ms/step - loss: 35196.0352 - val_loss: 20315.8145
Epoch 1113/10000
1/1 [==============================] - 0s 14ms/step - loss: 32255.1406 - val_loss: 19287.1738
E

Epoch 1179/10000
1/1 [==============================] - 0s 13ms/step - loss: 29065.3555 - val_loss: 20751.6465
Epoch 1180/10000
1/1 [==============================] - 0s 13ms/step - loss: 34728.2812 - val_loss: 23262.9590
Epoch 1181/10000
1/1 [==============================] - 0s 14ms/step - loss: 31467.1406 - val_loss: 21894.6973
Epoch 1182/10000
1/1 [==============================] - 0s 13ms/step - loss: 27028.3750 - val_loss: 21822.2109
Epoch 1183/10000
1/1 [==============================] - 0s 13ms/step - loss: 27172.7266 - val_loss: 20114.3594
Epoch 1184/10000
1/1 [==============================] - 0s 13ms/step - loss: 28976.5547 - val_loss: 20935.8633
Epoch 1185/10000
1/1 [==============================] - 0s 13ms/step - loss: 32270.5664 - val_loss: 19210.0840
Epoch 1186/10000
1/1 [==============================] - 0s 13ms/step - loss: 48299.1758 - val_loss: 20788.3398
Epoch 1187/10000
1/1 [==============================] - 0s 13ms/step - loss: 27865.9980 - val_loss: 20016.1660
E

Epoch 1253/10000
1/1 [==============================] - 0s 14ms/step - loss: 27824.1367 - val_loss: 14871.5557
Epoch 1254/10000
1/1 [==============================] - 0s 13ms/step - loss: 29306.2812 - val_loss: 14129.0713
Epoch 1255/10000
1/1 [==============================] - 0s 13ms/step - loss: 98422.5625 - val_loss: 14308.2275
Epoch 1256/10000
1/1 [==============================] - 0s 13ms/step - loss: 42914.8633 - val_loss: 16117.6865
Epoch 1257/10000
1/1 [==============================] - 0s 13ms/step - loss: 37152.9961 - val_loss: 15271.6396
Epoch 1258/10000
1/1 [==============================] - 0s 13ms/step - loss: 30262.5957 - val_loss: 15664.3486
Epoch 1259/10000
1/1 [==============================] - 0s 13ms/step - loss: 31790.2539 - val_loss: 14660.2705
Epoch 1260/10000
1/1 [==============================] - 0s 13ms/step - loss: 31195.0801 - val_loss: 16182.3242
Epoch 1261/10000
1/1 [==============================] - 0s 13ms/step - loss: 35118.6914 - val_loss: 20863.1895
E

Epoch 1327/10000
1/1 [==============================] - 0s 13ms/step - loss: 52938.8789 - val_loss: 62954.6680
Epoch 1328/10000
1/1 [==============================] - 0s 13ms/step - loss: 47904.0430 - val_loss: 66518.5859
Epoch 1329/10000
1/1 [==============================] - 0s 13ms/step - loss: 69864.8750 - val_loss: 61224.1719
Epoch 1330/10000
1/1 [==============================] - 0s 13ms/step - loss: 49670.1523 - val_loss: 70891.3359
Epoch 1331/10000
1/1 [==============================] - 0s 13ms/step - loss: 46720.5625 - val_loss: 71412.3203
Epoch 1332/10000
1/1 [==============================] - 0s 13ms/step - loss: 45674.4102 - val_loss: 74343.0234
Epoch 1333/10000
1/1 [==============================] - 0s 13ms/step - loss: 39206.8242 - val_loss: 71198.0938
Epoch 1334/10000
1/1 [==============================] - 0s 13ms/step - loss: 50016.8750 - val_loss: 69394.6328
Epoch 1335/10000
1/1 [==============================] - 0s 13ms/step - loss: 38540.5859 - val_loss: 67430.6172
E

1/1 [==============================] - 0s 13ms/step - loss: 61939.6250 - val_loss: 164614.9375
Epoch 1401/10000
1/1 [==============================] - 0s 13ms/step - loss: 71437.0703 - val_loss: 150515.4375
Epoch 1402/10000
1/1 [==============================] - 0s 13ms/step - loss: 62259.3398 - val_loss: 163610.2500
Epoch 1403/10000
1/1 [==============================] - 0s 13ms/step - loss: 59066.0391 - val_loss: 156996.2344
Epoch 1404/10000
1/1 [==============================] - 0s 13ms/step - loss: 85695.7656 - val_loss: 145231.8438
Epoch 1405/10000
1/1 [==============================] - 0s 13ms/step - loss: 63621.4766 - val_loss: 169345.4375
Epoch 1406/10000
1/1 [==============================] - 0s 13ms/step - loss: 63643.4180 - val_loss: 167813.2812
Epoch 1407/10000
1/1 [==============================] - 0s 13ms/step - loss: 73610.5859 - val_loss: 157031.4844
Epoch 1408/10000
1/1 [==============================] - 0s 13ms/step - loss: 83260.5156 - val_loss: 155099.7969
Epoch 140

In [19]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[4089.333](test_target) - [[2604.151]](prediction) = 1485.18212109375


In [20]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [21]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Valor Cimento Portland,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
0,-0.713127,1.867639,-1.651063,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-0.921973,-1.563415,-1.661916,-1.819268,-1.744222
1,-0.741803,1.018077,-1.103880,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-0.279547,-1.054585,-1.113781,-1.110478,-1.068353
2,-0.730617,0.721910,-0.595965,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-0.653845,-0.590157,-0.626867,-0.535461,-0.528616
3,-0.497967,-0.327566,-0.126085,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.757699,-0.152932,-0.022712,0.051840,-0.012460
4,-0.178038,-0.625198,0.306996,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,-0.254293,0.239109,0.585694,0.525953,0.401855
5,-0.003254,-0.972715,0.704512,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,-0.100172,0.569273,0.465685,0.821488,0.660648
6,0.426386,-0.622978,1.067697,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,0.591468,0.811321,0.850453,0.830229,0.749280
7,2.438420,-1.059170,1.397787,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,2.376061,1.741386,1.523443,1.235697,1.541868


train_target:


,Santa Catarina - Consumo de Cimento (t)
0,1740.105
1,1824.650
2,1870.478
3,2081.839
4,2653.587
5,2669.400
6,2852.759
7,3087.810


test_input:


...

,Santa Catarina - Produção de Cimento (t),Santa Catarina - Desemprego,Santa Catarina - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Santa Catarina - Valor Cimento Portland,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado
17,1.271362,-1.092682,-0.11741,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,1.917628,0.886688,-0.271847,-0.02587,0.728476


test_target:


,Santa Catarina - Consumo de Cimento (t)
17,4089.333


1/1 [==============================] - 0s 34ms/step
Error: 1485.18212109375




[2972.671875,
 3198.7548828125,
 3412.44091796875,
 3354.78125,
 3217.5087890625,
 3016.9814453125,
 2977.533203125,
 2977.6015625,
 3259.245361328125,
 2604.15087890625]

In [22]:
display(targets)
display(predictions)

[3268.31,
 3466.227,
 3246.749285714286,
 3017.084228571429,
 2776.217485714286,
 2891.489,
 2976.485,
 3258.027,
 3537.032,
 4089.333]

[2972.671875,
 3198.7548828125,
 3412.44091796875,
 3354.78125,
 3217.5087890625,
 3016.9814453125,
 2977.533203125,
 2977.6015625,
 3259.245361328125,
 2604.15087890625]

In [23]:
mae = mean_absolute_error(predictions, targets)
mae

367.7725044921874

In [24]:
porcentage = mae/np.mean(targets)
porcentage

0.11306699806326392